In [1]:
from typing import Optional, Union, Tuple, List, Callable, Dict
import os
import torch

import numpy as np
import random
import argparse
import json
from tqdm import tqdm

from diffusers import StableDiffusionPipeline, DDIMScheduler
from models.stylediffusion import global_var
from utils.utils import txt_draw
from PIL import Image

global_var._init()
global_var.set_value("USE_INITIAL_INV",False)
LOW_RESOURCE=True
global_var.set_value("LOW_RESOURCE",LOW_RESOURCE)
global_var.set_value("MAX_NUM_WORDS",77)
NUM_DDIM_STEPS = 50
global_var.set_value("NUM_DDIM_STEPS",NUM_DDIM_STEPS)
NUM_DDIM_STEPS_LCM = 8
global_var.set_value("NUM_DDIM_STEPS_LCM",NUM_DDIM_STEPS_LCM)
global_var.set_value("BLOCK_NUM",1)
global_var.set_value("IS_TRAIN",True)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device(
    'cpu')
global_var.set_value("device",device)
print(device)

# make the DDIM inversion pipeline
scheduler = DDIMScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", clip_sample=False, set_alpha_to_one=False)
stable = StableDiffusionPipeline.from_pretrained('runwayml/stable-diffusion-v1-5', scheduler=scheduler
, local_files_only=False).to(device)
tokenizer = stable.tokenizer
global_var.set_value("tokenizer",tokenizer)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.9/site-packages/accelerate/utils/torch_xla.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


cuda


Fetching 29 files:   0%|          | 0/29 [00:00<?, ?it/s]

/home/zeus/miniconda3/envs/cloudspace/lib/python3.9/site-packages/diffusers/pipelines/stable_diffusion/pipeline_stable_diffusion.py:101: FutureWarning: The configuration file of this scheduler: DDIMScheduler {
  "_class_name": "DDIMScheduler",
  "_diffusers_version": "0.10.0",
  "beta_end": 0.012,
  "beta_schedule": "scaled_linear",
  "beta_start": 0.00085,
  "clip_sample": false,
  "num_train_timesteps": 1000,
  "prediction_type": "epsilon",
  "set_alpha_to_one": false,
  "steps_offset": 0,
  "trained_betas": null
}
 is outdated. `steps_offset` should be set to 1 instead of 0. Please make sure to update the config accordingly as leaving `steps_offset` might led to incorrect results in future versions. If you have downloaded this checkpoint from the Hugging Face Hub, it would be very nice if you could open a Pull request for the `scheduler/scheduler_config.json` file
  deprecate("steps_offset!=1", "1.0.0", deprecation_message, standard_warn=False)


In [2]:
from models.stylediffusion.inversion import VaeInversion
#from models.stylediffusion.inversion_lcm import VaeInversionLCM
from models.stylediffusion.utils import  AttentionStore, EmptyControl, register_attention_control,make_controller
from models.stylediffusion import ptp_utils_v


def mask_decode(encoded_mask,image_shape=[512,512]):
    length=image_shape[0]*image_shape[1]
    mask_array=np.zeros((length,))

    for i in range(0,len(encoded_mask),2):
        splice_len=min(encoded_mask[i+1],length-encoded_mask[i])
        for j in range(splice_len):
            mask_array[encoded_mask[i]+j]=1

    mask_array=mask_array.reshape(image_shape[0], image_shape[1])
    # to avoid annotation errors in boundary
    mask_array[0,:]=1
    mask_array[-1,:]=1
    mask_array[:,0]=1
    mask_array[:,-1]=1

    return mask_array


# Infernce Code
@torch.no_grad()
def text2image_ldm_stable(
        model,
        prompt: List[str],
        trainer,
        controller,
        num_inference_steps: int = 50,
        guidance_scale: Optional[float] = 7.5,
        generator: Optional[torch.Generator] = None,
        latent: Optional[torch.FloatTensor] = None,
        start_time=50,
        return_type='image',
        use_fusion=False
):
    batch_size = len(prompt)
    register_attention_control(model, trainer, controller)
    height = width = 512
    
    if use_fusion:
        text_embeddings = model.text_encoder(prompt)
        uncond_embeddings = model.text_encoder([""]*batch_size)
    else:
        text_input = model.tokenizer(
            prompt,
            padding="max_length",
            max_length=model.tokenizer.model_max_length,
            truncation=True,
            return_tensors="pt",
        )
        text_embeddings = model.text_encoder(text_input.input_ids.to(model.device))[0]
        max_length = text_input.input_ids.shape[-1]
        uncond_input = model.tokenizer([""] * batch_size, padding="max_length", max_length=max_length, return_tensors="pt")
        uncond_embeddings = model.text_encoder(uncond_input.input_ids.to(model.device))[0]

    latent, latents = ptp_utils_v.init_latent(latent, model, height, width, generator, batch_size)
    # image_latents = [vae_inversion.latent2image(latents[0].unsqueeze(dim=0))[0]]
    model.scheduler.set_timesteps(num_inference_steps)
    for i, t in enumerate(tqdm(model.scheduler.timesteps[-start_time:])):
        trainer.I = i
        trainer.i = i \
            if i < NUM_DDIM_STEPS * trainer.v_replace_steps else None
        context = (uncond_embeddings, text_embeddings)
        latents = ptp_utils_v.diffusion_step(model, controller, latents, context, t, guidance_scale, low_resource=LOW_RESOURCE,)
        # image_latents += [vae_inversion.latent2image(latents[0].unsqueeze(dim=0))[0]]

    # os.makedirs('latent_save', exist_ok=True)
    # for i, latent_i in enumerate(image_latents):
    #     Image.fromarray(latent_i).save(f'latent_save/Z{NUM_DDIM_STEPS - i}_bar.png')

    if return_type == 'image':
        image = ptp_utils_v.latent2image(model.vae, latents)
    else:
        image = latents
    return image, latent

In [3]:
def run_and_display(stable, prompts, trainer, controller, latent=None, run_baseline=False, generator=None, verbose=True, use_fusion=False):
    if run_baseline:
        print("w.o. prompt-to-prompt")
        images, latent = run_and_display(stable, prompts, trainer, EmptyControl(), latent=latent, run_baseline=False,
                                         generator=generator, use_fusion=use_fusion)
        print("with prompt-to-prompt")
    images, x_t = text2image_ldm_stable(stable, prompts, trainer, controller, latent=latent,
                                        num_inference_steps=NUM_DDIM_STEPS, guidance_scale=global_var.get_value("GUIDANCE_SCALE"),
                                        generator=generator, use_fusion=use_fusion)
    if verbose:
        ptp_utils_v.view_images(images)
    return images, x_t



def setup_seed(seed=1234):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [4]:
def edit_image_stylediffusion_p2p(
    image_path,
    prompt_src,
    prompt_tar,
    guidance_scale=7.5,
    cross_replace_steps=0.4,
    self_replace_steps=0.6,
    blend_word=None,
    eq_params=None,
    is_replace_controller=False,
    num_inner_steps=100,
    num_epoch=1,
    tau_v=.6,
    tau_c=.6,
    tau_s=.8,
    tau_u=.5,
    use_fusion=False):
    global_var.set_value("GUIDANCE_SCALE",guidance_scale)
    global_var.set_value("IS_TRAIN",True)

    vae_inversion = VaeInversion(stable)
    (image_gt, image_rec), x_t, trainer = vae_inversion.invert(image_path, [prompt_src], verbose=True,
                                                                   num_inner_steps=num_inner_steps,
                                                                   num_epoch=num_epoch, use_fusion=use_fusion)
    print("Inversion complete")
    global_var.set_value("IS_TRAIN",False)

    trainer.attention_store = {}
    trainer.cur_step = 0
    # (image_gt, image_rec), x_t = vae_inversion.eval_init(image_path, [prompt_src], trainer=trainer)

    trainer.v_replace_steps = 1.0
    controller = AttentionStore()
    print("Doing reconstruction...")
    image_inv_recon, x_t_recon = run_and_display(stable, [prompt_src,prompt_tar], trainer, controller, run_baseline=False, latent=x_t,
                                             verbose=False, use_fusion=use_fusion)

    # edit
    trainer.v_replace_steps = tau_v
    cross_replace_steps = {'default_': tau_c,}
    self_replace_steps = tau_s
    uncond_self_replace_steps = tau_u
    controller = make_controller([prompt_src,prompt_tar],
                                 len(prompt_src.strip(" "))==len(prompt_tar.strip(" ")),
                                 cross_replace_steps,
                                 self_replace_steps,
                                 uncond_self_replace_steps,
                                 blend_word,
                                 eq_params)
    print("Doing editing...")
    image_inv_edit, x_t_edit = run_and_display(stable,
                                     [prompt_src,prompt_tar],
                                     trainer,
                                     controller,
                                     run_baseline=False,
                                     latent=x_t,
                                     verbose=False,
                                     use_fusion=use_fusion)

    image_instruct = txt_draw(f"source prompt: {prompt_src}\ntarget prompt: {prompt_tar}")

    out_image=Image.fromarray(np.concatenate((image_instruct,image_gt[0],image_inv_recon[0],image_inv_edit[1],),axis=1))

    return out_image

image_save_paths={
    "stylediffusion+p2p":"stylediffusion+p2p",
    }

In [5]:
def main_function(
    rerun_exist_images=False,
    data_path="data",
    output_path="output",
    edit_category_list=None,
    edit_method_list=None,
    use_fusion=False
):

    if edit_category_list is None:
        edit_category_list = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]

    if edit_method_list is None:
        edit_method_list = ["stylediffusion+p2p"]

    with open(f"{data_path}/mapping_file.json", "r") as f:
        editing_instruction = json.load(f)

        count = 0
        current_category = ""
        for key, item in editing_instruction.items():
            if item["editing_type_id"] not in edit_category_list:
                continue
            if current_category == item["editing_type_id"]:
                if count >= 20:
                    continue
                else:
                    count+=1
            else:
                current_category = item["editing_type_id"]
                count = 1

            original_prompt = item["original_prompt"].replace("[", "").replace("]", "")
            editing_prompt = item["editing_prompt"].replace("[", "").replace("]", "")
            image_path = os.path.join(f"{data_path}/annotation_images", item["image_path"])
            editing_instruction = item["editing_instruction"]
            blended_word = item["blended_word"].split(" ") if item["blended_word"] != "" else []
            mask = Image.fromarray(np.uint8(mask_decode(item["mask"])[:,:,np.newaxis].repeat(3,2))).convert("L")

            for edit_method in edit_method_list:
                present_image_save_path=image_path.replace(data_path, os.path.join(output_path,image_save_paths[edit_method]))
                if ((not os.path.exists(present_image_save_path)) or rerun_exist_images):
                    print(f"editing image [{image_path}] with [{edit_method}]")
                    setup_seed()
                    torch.cuda.empty_cache()
                    edited_image = edit_image_stylediffusion_p2p(
                            image_path=[image_path],
                            prompt_src=original_prompt,
                            prompt_tar=editing_prompt,
                            guidance_scale=7.5,
                            cross_replace_steps=0.4,
                            self_replace_steps=0.6,
                            blend_word=None,
                            eq_params=None,
                            is_replace_controller=False,
                            num_inner_steps=100,
                            num_epoch=1,
                            tau_v=.5,
                            tau_c=.6,
                            tau_s=.6,
                            tau_u=.0,
                            use_fusion=use_fusion) # disable p2pro

                    if not os.path.exists(os.path.dirname(present_image_save_path)):
                        os.makedirs(os.path.dirname(present_image_save_path))
                    edited_image.save(present_image_save_path)

                    print(f"finish")

                else:
                    print(f"skip image [{image_path}] with [{edit_method}]")

In [6]:
from transformers import T5Tokenizer, T5EncoderModel
import torch.nn as nn

class MultiEncoderFusion(nn.Module):

    # fusion of a T5 encoder and a clip encoder, following similar structure of double encoder used in SDXL.
    def __init__(self, clip_encoder, clip_tokenizer, t5_encoder, t5_tokenizer):
        super().__init__()
        self.clip_encoder = clip_encoder
        self.clip_tokenizer = clip_tokenizer
        for param in self.clip_encoder.parameters():
            param.requires_grad = False # freeze encoders
        self.t5_encoder = t5_encoder
        for param in self.t5_encoder.parameters():
            param.requires_grad = False
        self.t5_tokenizer = t5_tokenizer

        self.t5_projection = nn.Sequential(
            nn.Linear(512, 1024),
            nn.SiLU(),
            nn.Linear(1024, 768),
            nn.LayerNorm(768)
        )
        self.attn = nn.MultiheadAttention(768, num_heads=8, batch_first=True, dropout=0.1)
        self.fuse_projection = nn.Sequential(
            nn.Linear(1536, 1536),
            nn.SiLU(),
            nn.Dropout(0.1),
            nn.Linear(1536, 768),
        )
        self.layer_norm = nn.LayerNorm(768)

    def forward(self, prompt, device="cuda"):
        with torch.no_grad():
            # CLIP
            clip_tokens = self.clip_tokenizer(
                prompt, padding="max_length", max_length=77, truncation=True, return_tensors="pt"
            ).to(device)
            clip_emb = self.clip_encoder(clip_tokens.input_ids).last_hidden_state # [b,77,768]

            # T5
            t5_tokens = self.t5_tokenizer(
                prompt, return_tensors="pt", padding="max_length", max_length=128, truncation=True
            ).to(device)
            t5_emb = self.t5_encoder(t5_tokens.input_ids).last_hidden_state # [b,L,512]
        clip_emb = clip_emb.detach().clone()
        t5_emb = t5_emb.detach().clone()

        t5_proj = self.t5_projection(t5_emb) # [b,L,768]
        t5_aligned, _ = self.attn(query=clip_emb, key=t5_proj, value=t5_proj) # [b, 77, 768]

        fused = torch.cat([clip_emb, t5_aligned], dim=-1)  # [b,77,1536]
        fused = self.layer_norm(self.fuse_projection(fused)+clip_emb) # Projection to fit SD1.5 input dim

        return fused

In [7]:
%pip install sentencepiece==0.1.97

Note: you may need to restart the kernel to use updated packages.


In [8]:
fusion_state_dict = torch.load("fusion.pth")
t5_encoder = T5EncoderModel.from_pretrained("t5-small").eval()
t5_tokenizer = T5Tokenizer.from_pretrained("t5-small")

multifusion = MultiEncoderFusion(stable.text_encoder, stable.tokenizer, t5_encoder, t5_tokenizer).to(device)
multifusion.load_state_dict(fusion_state_dict, strict=False)
for p in multifusion.parameters():
    p.requires_grad = False

multifusion.eval()
stable.text_encoder = multifusion

Some weights of the model checkpoint at t5-small were not used when initializing T5EncoderModel: ['decoder.block.5.layer.1.EncDecAttention.q.weight', 'decoder.block.4.layer.0.SelfAttention.k.weight', 'decoder.block.1.layer.1.layer_norm.weight', 'decoder.block.0.layer.2.DenseReluDense.wi.weight', 'decoder.block.3.layer.1.EncDecAttention.k.weight', 'decoder.block.3.layer.2.DenseReluDense.wo.weight', 'decoder.block.4.layer.2.DenseReluDense.wo.weight', 'decoder.block.3.layer.0.SelfAttention.o.weight', 'decoder.block.4.layer.2.DenseReluDense.wi.weight', 'decoder.block.1.layer.1.EncDecAttention.q.weight', 'decoder.block.3.layer.2.DenseReluDense.wi.weight', 'decoder.block.0.layer.1.EncDecAttention.k.weight', 'decoder.block.2.layer.2.DenseReluDense.wi.weight', 'decoder.block.2.layer.1.EncDecAttention.k.weight', 'decoder.block.3.layer.0.layer_norm.weight', 'decoder.block.2.layer.0.SelfAttention.v.weight', 'decoder.block.3.layer.1.EncDecAttention.v.weight', 'decoder.block.4.layer.1.EncDecAttenti

In [ ]:
main_function(output_path="output", use_fusion=True)

skip image [data/annotation_images/0_random_140/000000000000.jpg] with [stylediffusion+p2p]
skip image [data/annotation_images/0_random_140/000000000001.jpg] with [stylediffusion+p2p]
skip image [data/annotation_images/0_random_140/000000000002.jpg] with [stylediffusion+p2p]
skip image [data/annotation_images/0_random_140/000000000003.jpg] with [stylediffusion+p2p]
skip image [data/annotation_images/0_random_140/000000000004.jpg] with [stylediffusion+p2p]
skip image [data/annotation_images/0_random_140/000000000005.jpg] with [stylediffusion+p2p]
skip image [data/annotation_images/0_random_140/000000000006.jpg] with [stylediffusion+p2p]
skip image [data/annotation_images/0_random_140/000000000007.jpg] with [stylediffusion+p2p]
skip image [data/annotation_images/0_random_140/000000000008.jpg] with [stylediffusion+p2p]
skip image [data/annotation_images/0_random_140/000000000009.jpg] with [stylediffusion+p2p]
skip image [data/annotation_images/0_random_140/000000000010.jpg] with [styledif

Epoch[1/1, t=49, iter=1]: 100%|████████████████████| 1071/1071 [09:54<00:00,  1.80it/s, loss=0.0898]


Inversion complete
Doing reconstruction...


100%|██████████| 50/50 [00:24<00:00,  2.05it/s]


Doing editing...


...

100%|██████████| 50/50 [00:23<00:00,  2.09it/s]


Doing editing...


100%|██████████| 50/50 [00:21<00:00,  2.34it/s]


finish
editing image [data/annotation_images/9_change_style_80/1_artificial/2_human/912000000002.jpg] with [stylediffusion+p2p]
DDIM inversion...
StyleDiffusion optimization...


Epoch[1/1, t=0, iter=100]:   2%|▍                   | 21/1071 [00:11<09:37,  1.82it/s, loss=7.75e-5]

In [10]:
!PYTHONPATH=/teamspace/studios/this_studio/CIS-6450-Final-Project python evaluation/evaluate.py --metrics "lpips" "ssim" "lpips_unedit_part" "ssim_unedit_part" "clip_similarity_source_image" "clip_similarity_target_image" "clip_similarity_target_image_edit_part" --result_path evaluation_result_sd.csv --edit_category_list 0 1 2 3 4 5 6 7 8 9 --tgt_methods 1_stylediffusion+p2p

/home/zeus/miniconda3/envs/cloudspace/lib/python3.9/site-packages/torchmetrics/utilities/imports.py:22: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution
/home/zeus/miniconda3/envs/cloudspace/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=SqueezeNet1_1_Weights.IMAGENET1K_V1`. You can also use `weights=